In [1]:
import pandas as pd
import numpy as np
import os, sys, time
import yaml
from sklearn.metrics import matthews_corrcoef

%matplotlib inline  
import pandas as pd
import numpy as np
import os, sys, time
import yaml

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils
from utils.bosch_functions import load_processed_bosch_data
from utils.models import CombinedModel, ExtraTreeModel
from utils.validation_tools import score_MCC, create_validation_index, cross_validate_model


In [2]:
from utils.wrapped_xgboost import xgboost_classifier

In [2]:
'''
project_path = '/mnt/home/ymm/kaggle/compete/current'
data_path = '/mnt/home/ymm/kaggle/bosch_processed_data'

old way to import data
yaml_file = 'bosch_processed_data_dict.yml'
with open(os.path.join(project_path, yaml_file), 'r') as yml_stream:
    data_dict = yaml.load(yml_stream)

data_index = '0'
data_file = os.path.join(data_path, data_dict[data_index]['train_file'])
print 'loading data from ', data_file
train = pd.read_csv(data_file, index_col='Id')
'''

"\nproject_path = '/mnt/home/ymm/kaggle/compete/current'\ndata_path = '/mnt/home/ymm/kaggle/bosch_processed_data'\n\nold way to import data\nyaml_file = 'bosch_processed_data_dict.yml'\nwith open(os.path.join(project_path, yaml_file), 'r') as yml_stream:\n    data_dict = yaml.load(yml_stream)\n\ndata_index = '0'\ndata_file = os.path.join(data_path, data_dict[data_index]['train_file'])\nprint 'loading data from ', data_file\ntrain = pd.read_csv(data_file, index_col='Id')\n"

In [3]:
## load training data
#project_path = '/mnt/home/ymm/kaggle/compete/current'
#data_path = '/mnt/home/ymm/kaggle/bosch_data/bosch_complete_processed_data'
#data_yaml_file = 'bosch_processed_data_dict.yml'

project_path = '/home/ymm/kaggle/compete/current/model_6_bins_data'
data_path = '/home/ymm/kaggle/bosch_data/bosch_complete_processed_6_bins_data'
data_yaml_file = 'complete_subset_data_6_bins_dict.yml'

dep_var_name = 'Response'
data_index= '0'
train = load_processed_bosch_data(data_path, project_path, data_yaml_file, data_index=data_index)
train_index, valid_index = create_validation_index(train, 0.5, dep_var_name, True)  
valid_data = train.ix[valid_index]
tmp_train  = train.ix[train_index]

loading bosch data from  /home/ymm/kaggle/bosch_data/bosch_complete_processed_6_bins_data/processed_totBins_6_bin_0_train.csv


In [18]:
reload(utils)
reload(utils.wrapped_xgboost)
#dir(utils.wrapped_xgboost)
#reload(utils.validation_tools)

<module 'utils.wrapped_xgboost' from '/home/ymm/kaggle/xgboost_hyperopt/utils/wrapped_xgboost.pyc'>

In [5]:
#reload(utils.wrapped_xgboost)

In [147]:
'''
from utils.validation_tools import create_validation_index
## obtein the index for train and validation data
train_index, valid_index = create_validation_index(train, dep_var_name='Response', valid_frac=0.3)
'''

"\nfrom utils.validation_tools import create_validation_index\n## obtein the index for train and validation data\ntrain_index, valid_index = create_validation_index(train, dep_var_name='Response', valid_frac=0.3)\n"

### xgboost for imbalanced data

#### data preparation
    a. use the larger dataset, first bin out of 6-bins data
    b. a fully processed data, missing values are already filled

#### model benchmark
    params["eta"]                      = 0.01
    params["subsample"]                = 0.8
    params["colsample_bytree"]         = 0.8
    params["num_round"]                = 500

#### approaches
    1. Use param 'scale_pos_weight', as the ratio of negative samples to positive ones
     3-folds results: [0.17132387528893958, 0.18898085550009494, 0.15096433310166174]
     
    2. valina xgboost without any additional modification.
     3-folds results: 
    
    2. Assign proper weights to samples

In [4]:
scale_pos_weight = 1. * np.sum(train[dep_var_name].values == 0) / np.sum(train[dep_var_name].values == 1)
print 'scale_pos_weight:', scale_pos_weight

init_weight = np.copy(train[dep_var_name].values)

np.place(init_weight, init_weight==1, scale_pos_weight)
np.place(init_weight, init_weight==0, 1.)

init_weight = init_weight.astype(float)
#init_weight[:500]

scale_pos_weight: 187.502097902


In [22]:
dep_var_name = 'Response'

params = {}
params["eta"]                      = 0.01
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 1000
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 100
params["val"]                      = False
params["early_stopping_ratio"]     = 0.2
## important param for highly imbalanced data
#params['scale_pos_weight']         = scale_pos_weight

In [23]:
xgb_clf = xgboost_classifier(label_name = dep_var_name, params = params)

In [ ]:
results = xgb_clf.cross_validate_fit(score_MCC, train, use_weights = True, n_folds=3)

scale_pos_weight: 187.764705882



####################
 train the xgboost without early stopping
####################


weights are used for xgboost training data...
[0]	train-auc:0.757654
[1]	train-auc:0.790898
[2]	train-auc:0.805074
[3]	train-auc:0.809885
[4]	train-auc:0.810948
[5]	train-auc:0.814427
[6]	train-auc:0.818645
[7]	train-auc:0.82283
[8]	train-auc:0.825488
[9]	train-auc:0.825008
[10]	train-auc:0.829774
[11]	train-auc:0.830754
[12]	train-auc:0.832469
[13]	train-auc:0.833765
[14]	train-auc:0.83544
[15]	train-auc:0.83665
[16]	train-auc:0.838642
[17]	train-auc:0.839358
[18]	train-auc:0.839225
[19]	train-auc:0.840876
[20]	train-auc:0.841949
[21]	train-auc:0.842047
[22]	train-auc:0.842563
[23]	train-auc:0.842802
[24]	train-auc:0.844478
[25]	train-auc:0.846359
[26]	train-auc:0.847522
[27]	train-auc:0.848715
[28]	train-auc:0.848558
[29]	train-auc:0.849879
[30]	train-auc:0.850941
[31]	train-auc:0.851296
[32]	train-auc:0.852558
[33]	train-auc:0.852526
[34]	train-auc:0.854047
[35]	train-auc:0.854946
[36]	train-auc:0.856217
[37]	train-auc:0.856819
[38]	train-auc:0.85783
[39]	train-auc:0.858435
[40]	tra

In [25]:
print results

[0.12925909738482472, 0.16786056527874313, 0.17630868136728384]


### single xgb model

In [141]:
reload(utils.validation_tools)
dir(utils.validation_tools)

['StratifiedKFold',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 'create_validation_index',
 'cross_validate_model',
 'matthews_corrcoef',
 'np',
 'os',
 'pd',
 'score_MCC',
 'shuffle',
 'sys',
 'time']

In [38]:
xgb_train_data = train.ix[train_index]
xgb_test_data = train.ix[valid_index]

In [51]:
xgb_clf = xgboost_classifier(xgb_train_data, dep_var_name, params = params)
xgb_clf.fit()


####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.505492
[1]	train-auc:0.505492
[2]	train-auc:0.516412
[3]	train-auc:0.516412
[4]	train-auc:0.527302
[5]	train-auc:0.527272
[6]	train-auc:0.591767
[7]	train-auc:0.591868
[8]	train-auc:0.59186
[9]	train-auc:0.591876
[10]	train-auc:0.591876
[11]	train-auc:0.591877
[12]	train-auc:0.591864
[13]	train-auc:0.591861
[14]	train-auc:0.597336
[15]	train-auc:0.597341
[16]	train-auc:0.597227
[17]	train-auc:0.597221
[18]	train-auc:0.597218
[19]	train-auc:0.59719
[20]	train-auc:0.59719
[21]	train-auc:0.597188
[22]	train-auc:0.597188
[23]	train-auc:0.597193
[24]	train-auc:0.597196
[25]	train-auc:0.5972
[26]	train-auc:0.602639
[27]	train-auc:0.602647
[28]	train-auc:0.602645
[29]	train-auc:0.602644
[30]	train-auc:0.602629
[31]	train-auc:0.602621
[32]	train-auc:0.602626
[33]	train-auc:0.602624
[34]	train-auc:0.602626
[35]	train-auc:0.602627
[36]	train-auc:0.602626
[37]	train-auc:0.602626
[38]	train-auc:0.602626
[39]	train-auc:0.602597
[40]	train-auc:0.602596
[41]	train-auc:0.602598
[42]	tr

In [41]:
pred_res = xgb_clf.predict(xgb_test_data)
validation_tools.score_MCC(xgb_test_data['Response'].values, pred_res)

In [43]:
sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.validation_tools as validation_tools

In [47]:
reload(validation_tools)
dir(validation_tools)

<module 'utils.validation_tools' from '/home/ymm/kaggle/xgboost_hyperopt/utils/validation_tools.py'>